In [0]:
import pandas as pd
import random
import json
import numpy as np 
from entity_classifier_model import EntityIdentifier
from metrics import results_analyser

In [0]:
Current_path =  #your current path

In [0]:
def read_results(results_path):
  #reads the test results of NER model
  #results are saved as a json file, each item is realted to one sentence
  #keys are true labels (list of IOB), predicted labels (list of IOB) and tokenized text 
  with open(results_path, 'r') as f_:
    data = json.load(f_)

  y_true = [item['true_label'] for item in data]
  y_pred = [item['predicted_label'] for item in data]
  sentences = [item['sent'] for item in data]
  
  return y_true, y_pred, sentences
    

#Load st21pv entity classifier

In [0]:
stpv_model_loaded = EntityIdentifier(num_labels=22)  
stpv_model_loaded.load_model(Current_path + 'entity_identifier_models/st21pv/')
precisions = stpv_model_loaded.precisions

In [0]:
precisions

array([0.72081081, 0.84347826, 0.81263307, 0.68472906, 0.74660633,
       0.8115942 , 0.78476616, 0.67346939, 0.82532504, 0.74537037,
       0.69565217, 0.80412371, 0.83823529, 0.7983154 , 0.65337423,
       0.79736842, 0.92053501, 0.92243767, 0.88099267, 0.80726257,
       0.74611399, 0.77339612])

##Read results of biobert+linear on the test set of st21pv

In [0]:
stpv_biobert_results_path = Current_path +'st21pv_NER_results/stpv_biobert_lstm_results.json'
y_true, y_pred, sentences = read_results(stpv_biobert_results_path)

In [0]:
y_true[0],y_pred[0], sentences[0]

(['O',
  'O',
  'B-research_activity',
  'O',
  'O',
  'O',
  'B-health_care_activity',
  'I-health_care_activity',
  'I-health_care_activity',
  'O',
  'O',
  'B-anatomical_structure',
  'I-anatomical_structure',
  'O',
  'O',
  'O',
  'B-chemical',
  'I-chemical',
  'I-chemical',
  'I-chemical',
  'O',
  'B-chemical',
  'O'],
 ['O',
  'O',
  'B-research_activity',
  'O',
  'O',
  'O',
  'B-intellectual_product',
  'I-intellectual_product',
  'I-intellectual_product',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-intellectual_product',
  'I-intellectual_product',
  'I-intellectual_product',
  'I-intellectual_product',
  'O',
  'B-intellectual_product',
  'O'],
 ['Development',
  'and',
  'validation',
  'of',
  'an',
  'easy-to-use',
  'risk',
  'assessment',
  'tool',
  'for',
  'cumulative',
  'low',
  'back',
  'loading',
  ':',
  'The',
  'Lifting',
  'Fatigue',
  'Failure',
  'Tool',
  '(',
  'LiFFT',
  ')'])

##Error Analysis and F-score

In [0]:
stpv_biobert_analyser =results_analyser(y_true,y_pred,sentences, stpv_model_loaded,precisions)
Fscores = stpv_biobert_analyser.get_all_fscores()

In [0]:
print('Excat Fscore = ', Fscores[0])
print('Learning-Based Fscoer = ', Fscores[1])
print('Relaxed Fscoer = ', Fscores[2])

Excat Fscore =  0.6323408416107222
Learning-Based Fscoer =  0.7350082873257575
Relaxed Fscoer =  0.7586420709967446


In [0]:
print('number of exact matches = ', stpv_biobert_analyser.n_correct)
print('number of false positives = ', stpv_biobert_analyser.n_false_positive )
print('number of false negatives = ', stpv_biobert_analyser.n_false_negative)
print('number of extracted entities with wrong label and right span =', stpv_biobert_analyser.n_wrong_label_right_span)
print('number of extracted entities with wrong label and overlapping span =', stpv_biobert_analyser.n_wrong_label_over_span)
print('number of extracted entities with right label and overlapping span =',stpv_biobert_analyser.n_right_label_over_span)
print('number of accepted Type_5 mismatches = ',  stpv_biobert_analyser.get_overlap_score())
print('number of rejected_Type_5 mismatches = ', stpv_biobert_analyser.n_right_label_over_span- stpv_biobert_analyser.get_overlap_score())

number of exact matches =  26515
number of false positives =  6752
number of false negatives =  3977
number of extracted entities with wrong label and right span = 3145
number of extracted entities with wrong label and overlapping span = 3100
number of extracted entities with right label and overlapping span = 5296



number of accepted Type_5 mismatches =  4305



number of rejected_Type_5 mismatches =  991
